## Simulação

In [ ]:
import importlib
import arbitragem

importlib.reload(arbitragem)

data_inicio = '2024-06-20'
data_fim   = '2025-07-19'

janelas= [500,1000,1500,2000,2500,3000,3500,4000,45000]

retorno_pct, retorno_risco, sharpe = arbitragem.simular_estrategia(
    moeda1='BTC',
    moeda2='BNB',
    zscore_compra_e_venda=2.0,
    zscore_encerrar_posicao=0.5,
    stop_loss=3.0,
    cooldown_stop_loss=20,
    janela=i,
    data_inicial='2025-06-20',
    data_final='2025-07-19'
    )
print(f"Simulação Janela: {i}")
print(f"Retorno percentual: {retorno_pct*100:.2f}%")
print(f"Retorno ajustado ao risco: {retorno_risco:.4f}")
print(f"Índice de Sharpe: {sharpe:.4f}")





Simulação Janela: 500
Retorno percentual: -28.69%
Retorno ajustado ao risco: -25.7670
Índice de Sharpe: -4.2303
Simulação Janela: 1000
Retorno percentual: -23.29%
Retorno ajustado ao risco: -10.5081
Índice de Sharpe: -2.3207
Simulação Janela: 1500
Retorno percentual: -15.09%
Retorno ajustado ao risco: -5.6799
Índice de Sharpe: -1.9880
Simulação Janela: 2000
Retorno percentual: -12.11%
Retorno ajustado ao risco: -3.6538
Índice de Sharpe: -1.8770
Simulação Janela: 2500
Retorno percentual: -11.26%
Retorno ajustado ao risco: -4.1918
Índice de Sharpe: -2.1513
Simulação Janela: 3000
Retorno percentual: -2.37%
Retorno ajustado ao risco: -0.8994
Índice de Sharpe: -0.4457
Simulação Janela: 3500
Retorno percentual: 0.44%
Retorno ajustado ao risco: nan
Índice de Sharpe: nan
Simulação Janela: 4000
Retorno percentual: -2.48%
Retorno ajustado ao risco: nan
Índice de Sharpe: nan
Simulação Janela: 45000
Retorno percentual: 0.00%
Retorno ajustado ao risco: 0.0000
Índice de Sharpe: 0.0000


In [5]:
from sklearn.model_selection import ParameterGrid, TimeSeriesSplit
import pandas as pd
import numpy as np

import importlib
import arbitragem

importlib.reload(arbitragem)
lista_moedas = ['BTC', 'BNB']
intervalo = "1d"

m1 =lista_moedas[0]
m2 =lista_moedas[1]

def validacao_cruzada_temporal(df, param_grid, n_splits=5):
    """
    Executa uma validação cruzada temporal para encontrar os melhores parâmetros.

    Parâmetros:
    - df: DataFrame com os dados de preços (deve conter Ativo1 e Ativo2)
    - param_grid: dicionário com os parâmetros a otimizar
    - n_splits: número de divisões temporais

    Retorna:
    - melhores_parametros: dicionário com os melhores parâmetros
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    melhores_parametros = None
    melhor_score = -np.inf

    resultados = []

    for params in ParameterGrid(param_grid):
        scores = []

        for train_idx, val_idx in tscv.split(df):
            df_train = df.iloc[train_idx]
            df_val = df.iloc[val_idx]

            try:
                retorno_pct, _, _ = arbitragem.simular_estrategia(
                    moeda1=m1,
                    moeda2=m2,
                    zscore_compra_e_venda=params['zscore_compra_venda'],
                    zscore_encerrar_posicao=params['zscore_encerrar_posicao'],
                    stop_loss=0.0,
                    cooldown_stop_loss=0,
                    janela=params['janela'],
                    data_inicial=df_train.index[0].strftime('%Y-%m-%d'),
                    data_final=df_train.index[-1].strftime('%Y-%m-%d'),
                    taxa=0.01
                )

                if retorno_pct is not None:
                    scores.append(retorno_pct)
            except Exception as e:
                continue

        media_retorno = np.mean(scores) if scores else -np.inf
        resultados.append((params, media_retorno))

        if media_retorno > melhor_score:
            melhor_score = media_retorno
            melhores_parametros = params

    return melhores_parametros, pd.DataFrame(resultados, columns=["params", "retorno_medio"])


# Dados entre 2024-06-20 e 2025-06-19
df_train_total = arbitragem.carregar_dados(m1, m2).loc['2024-06-20':'2025-04-19']

# Defina a grade de parâmetros
param_grid = {
    'zscore_compra_venda': [1.5, 2.0, 2.5],
    'zscore_encerrar_posicao': [0, 0.2, 0.3, 0.5,],
    'janela': [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25,30]
}

melhores_parametros, df_resultados = validacao_cruzada_temporal(df_train_total, param_grid, n_splits=4)

print("Melhores parâmetros:", melhores_parametros)
print(df_resultados.sort_values("retorno_medio", ascending=False))

df_resultados.to_excel(f"resultados/otimizacao/{intervalo}/{m1}X{m2} - Resultado Validacao Cruzada.xlsx", index=False)

# Validação final
retorno_val, risco_val, sharpe_val = arbitragem.simular_estrategia(
    moeda1=m1,
    moeda2=m2,
    zscore_compra_e_venda=melhores_parametros['zscore_compra_venda'],
    zscore_encerrar_posicao=melhores_parametros['zscore_encerrar_posicao'],
    stop_loss=0.0,
    cooldown_stop_loss=0,
    janela=melhores_parametros['janela'],
    data_inicial='2025-05-20',
    data_final='2025-07-19',
    periodo_observacoes=intervalo,
    taxa=0.01,
    capital_inicial=10000
)

print(f"\nValidação final:")
print(f"Retorno: {retorno_val*100:.2f}% | Sharpe: {sharpe_val:.2f}")

Melhores parâmetros: {'janela': 11, 'zscore_compra_venda': 2.5, 'zscore_encerrar_posicao': 0.5}
                                                params  retorno_medio
95   {'janela': 11, 'zscore_compra_venda': 2.5, 'zs...       0.211079
83   {'janela': 10, 'zscore_compra_venda': 2.5, 'zs...       0.192110
3    {'janela': 4, 'zscore_compra_venda': 1.5, 'zsc...       0.187521
7    {'janela': 4, 'zscore_compra_venda': 2.0, 'zsc...       0.187521
11   {'janela': 4, 'zscore_compra_venda': 2.5, 'zsc...       0.187521
..                                                 ...            ...
173  {'janela': 18, 'zscore_compra_venda': 2.0, 'zs...      -0.359509
169  {'janela': 18, 'zscore_compra_venda': 1.5, 'zs...      -0.369404
26   {'janela': 6, 'zscore_compra_venda': 1.5, 'zsc...      -0.369536
172  {'janela': 18, 'zscore_compra_venda': 2.0, 'zs...      -0.396361
168  {'janela': 18, 'zscore_compra_venda': 1.5, 'zs...      -0.412009

[228 rows x 2 columns]

Validação final:
Retorno: 2.76% | Sharp

In [2]:
pares_cointegrados = [
    ["BTC","BNB"],
    ["BNB", "XRP"],
]

for cada_par in pares_cointegrados:
    #RODAR SIMULACAO
    print(cada_par)

['BTC', 'BNB']
['BNB', 'XRP']


## Teste Manual

In [ ]:

# Algoritmo de otimizacao
zscv = 1
zse = 0.5

stop_loss = 0

cooldown = 0
janela = 60

dfcodigo = carregar_dados("ETH", "WBTC")

# Intervalo para teste de cointegração
data_inicial = '2025-06-20'
data_final   = '2025-07-19'
df_filtrado = dfcodigo.loc[data_inicial:data_final]


df_sinalizado = gerar_sinais(df_filtrado, zscv, zse, stop_loss, cooldown, janela) #gera os sinais de compra_venda_encerramento

capital_final, retorno_percentual, capital_series, df_com_retorno_financeiro = simular_retorno_por_trade(df_sinalizado) #simula os retornos

#Valores pra gráfico
df_com_retorno_financeiro['ativo1_valor_esperado'] = df_com_retorno_financeiro['Ativo2'] + df_com_retorno_financeiro['rolling_mean']
df_com_retorno_financeiro['ativo1_desvio_preco'] = df_com_retorno_financeiro['ativo1_valor_esperado'] - df_com_retorno_financeiro['Ativo1']
df_com_retorno_financeiro['ativo1_std_preco'] = df_com_retorno_financeiro['ativo1_desvio_preco'] / df_com_retorno_financeiro['rolling_std']

df_com_retorno_financeiro['ativo2_valor_esperado'] = df_com_retorno_financeiro['Ativo1'] - df_com_retorno_financeiro['rolling_mean']
df_com_retorno_financeiro['ativo2_desvio_preco'] = df_com_retorno_financeiro['ativo2_valor_esperado'] - df_com_retorno_financeiro['Ativo2']
df_com_retorno_financeiro['ativo2_std_preco'] = df_com_retorno_financeiro['ativo2_desvio_preco'] / df_com_retorno_financeiro['rolling_std']


df_com_retorno_financeiro.to_excel('retornos.xlsx', index=False)

print(f"\nRetorno acumulado: {retorno_percentual * 100:.2f}%")
print(f"Capital final: ${capital_final:.2f}")
